# Segment spheroids for Fgure 4b
Data: reconstructed 3D green channel images of MDCK spheroids

## Libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from skimage.measure import regionprops_table, marching_cubes, mesh_surface_area
from skimage.color import label2rgb
from skimage.morphology import remove_small_objects
from skimage.segmentation import clear_border

import pyclesperanto_prototype as cle

## Functions

In [3]:
def segment(data, suffix, sigma_spot_detection = 8, sigma_outline = 2):
    '''
    Segment spheroids

    data: name of the input data
    suffix: suffix of the input data
    sigma_spot_detectio: sigma of Gaussian filter for detecting spheroids
    sigma_outline: sigmae of Gaussian fileter for detecting edges of spheroids    
    '''
    res = []
    for s in suffix:
        print(s)
        img = np.load(f'../result/{data}{s}.npy')
        img = img[25:75, 10:630, 45:img.shape[2]-50]
        img_gpu = cle.push(img)
        segmented = cle.voronoi_otsu_labeling(img_gpu, spot_sigma=sigma_spot_detection, outline_sigma=sigma_outline)
        res.append(cle.pull(segmented))

    return res

In [5]:
def calculate_props(segmented, suffix, scale=(None, None, None)):
    '''
    Calculate morphological properties of spheroids

    segmented: label images after segmentation
    suffix: suffix of the input data    
    scale: pixel resolution in um scale
    '''
    res = []
    surface = []
    
    for i, s_data in enumerate(segmented):
        print(suffix[i])
        s_data = clear_border(s_data) # remove objects on the edge
        s_data = remove_small_objects(s_data, min_size=100) # remove small objects

        # calculate properties
        props = pd.DataFrame(regionprops_table(label_image=s_data, intensity_image=intensity_img,
                                               properties=('label', 'area', 'solidity', 'bbox'), spacing=scale))
        props = props.rename(columns={'label':'Label', 'area': 'Volume', 'solidity': 'Solidity'})
        res.append(props)
        
    return res

In [1]:
def filter(segmented, props, data, suffix, volume_thresh=20000, solidity_thresh=0.8):
    '''
    Filter out spheroids without beads

    segmented: label images after segmentation
    props: calculated properties
    data: name of the input data
    suffix: suffix of the input data  
    volume_thresh: threshold value of volume
    splidity_thresh: threshold value of solidity
    '''
    for i, segment in enumerate(segmented):
        props_filtered = props[i][(props[i]['Volume']>volume_thresh)&(props[i]['Solidity']>solidity_thresh)]
        retained_label = props_filtered['Label'].values
        label_img = np.where(np.isin(segment, retained_label), segment, 0)
        np.save(f'../result/{data}{suffix[i]}_segmented_spheroids.npy', label_img)

## Parameters

In [3]:
suffix = ['', '_X1', '_X2', '_X3', '_X4', '_X5', '_X6']
scale = np.repeat(0.295*4, 3)

NameError: name 'np' is not defined

## Analysis

### Measurement: 1

In [7]:
data = 'id551_MDCK_unit_on_on_01_300Hz_405_250mW_488_150mW_488'

In [8]:
res_segment = segment(data, suffix)


size: 50
_X1
size: 50
_X2
size: 50
_X3
size: 50
_X4
size: 50
_X5
size: 50
_X6
size: 50


In [10]:
res_props = calculate_props(res_segment, suffix, scale=scale)


_X1
_X2
_X3
_X4
_X5
_X6


In [11]:
filter(res_segment, res_props, data, suffix)

In [12]:
for i in range(len(res_segment)):
    print(res_segment[i].shape)

(50, 620, 2687)
(50, 620, 2687)
(50, 620, 2687)
(50, 620, 2687)
(50, 620, 2687)
(50, 620, 2687)
(50, 620, 1151)


### Measurement: 2

In [7]:
data = 'id551_MDCK_unit_on_on_02_300Hz_405_250mW_488_150mW_488'

In [8]:
res_segment = segment(data, suffix)


size: 50
_X1
size: 50
_X2
size: 50
_X3
size: 50
_X4
size: 50
_X5
size: 50
_X6
size: 50


In [10]:
res_props = calculate_props(res_segment, suffix, scale=scale)


_X1
_X2
_X3
_X4
_X5
_X6


In [11]:
filter(res_segment, res_props, data, suffix)

In [12]:
for i in range(len(res_segment)):
    print(res_segment[i].shape)

(50, 620, 2687)
(50, 620, 2687)
(50, 620, 2687)
(50, 620, 2687)
(50, 620, 2687)
(50, 620, 2687)
(50, 620, 1151)


### Measurement: 3

In [13]:
data = 'id551_MDCK_unit_on_on_03_300Hz_405_250mW_488_150mW_488'

In [14]:
res_segment = segment(data, suffix)


size: 50
_X1
size: 50
_X2
size: 50
_X3
size: 50
_X4
size: 50
_X5
size: 50
_X6
size: 50


In [16]:
res_props = calculate_props(res_segment, suffix, scale=scale)


_X1
_X2
_X3
_X4
_X5
_X6


In [17]:
filter(res_segment, res_props, data, suffix)

In [18]:
for i in range(len(res_segment)):
    print(res_segment[i].shape)

(50, 620, 2723)
(50, 620, 2723)
(50, 620, 2723)
(50, 620, 2723)
(50, 620, 2723)
(50, 620, 2723)
(50, 620, 1166)


## Calculate the number of spheroids

In [25]:
data = 'id551_MDCK_unit_on_on_01_300Hz_405_250mW_488_150mW_488'

In [26]:
total = []
for i, s in enumerate(suffix):
    spheroid = np.load(f'../result/{data}{s}_segmented_spheroids.npy')
    num = len(np.unique(spheroid))-1 
    print(num)
    total.append(num)
print(sum(total))

65
72
87
87
70
47
28
456


In [19]:
data = 'id551_MDCK_unit_on_on_02_300Hz_405_250mW_488_150mW_488'

In [22]:
total = []
for i, s in enumerate(suffix):
    spheroid = np.load(f'../result/{data}{s}_segmented_spheroids.npy')
    num = len(np.unique(spheroid))-1 
    print(num)
    total.append(num)
print(sum(total))

33
42
59
50
56
53
19
312


In [23]:
data = 'id551_MDCK_unit_on_on_03_300Hz_405_250mW_488_150mW_488'

In [24]:
total = []
for i, s in enumerate(suffix):
    spheroid = np.load(f'../result/{data}{s}_segmented_spheroids.npy')
    num = len(np.unique(spheroid))-1 
    print(num)
    total.append(num)
print(sum(total))

44
35
25
23
23
26
10
186


In [27]:
456+312+186

954